In [ ]:
import yfinance as yf
import pandas as pd
import os
import numpy as np
import time
import math
from datetime import datetime

# Load stock symbols and their listing dates from CSV
stock_list_path = "NS.csv"  # Path to your CSV
stock_df = pd.read_csv(stock_list_path)
stock_df['DATE OF LISTING'] = pd.to_datetime(stock_df['DATE OF LISTING'])

# Create a directory to store output files
output_directory = "S-Curve_Data"
os.makedirs(output_directory, exist_ok=True)

# ---------------------- Rate-Limited Download Functions ----------------------
def safe_download(symbol, start, retries=3, delay=60):
    """Download stock data with rate limit handling"""
    for attempt in range(retries):
        try:
            data = yf.download(symbol, start=start, interval='1d', progress=False)
            if not data.empty:
                return data
        except Exception as e:
            if 'Too Many Requests' in str(e) or 'Rate limited' in str(e):
                print(f"⏳ Rate limit hit for {symbol}. Waiting {delay} seconds (Attempt {attempt+1}/{retries})...")
                time.sleep(delay)
            else:
                print(f"❌ Unexpected error downloading {symbol}: {e}")
                break
    return pd.DataFrame()

def safe_info(ticker_obj, retries=3, delay=30):
    """Fetch ticker info with rate limit handling"""
    for attempt in range(retries):
        try:
            return ticker_obj.info
        except Exception as e:
            if 'Too Many Requests' in str(e) or 'Rate limited' in str(e):
                print(f"⏳ Rate limit (info) hit. Waiting {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"❌ Info error: {e}")
                break
    return {}

def detect_s_curve(price_series, years):
    """
    Enhanced S-curve detection algorithm that identifies the start of sustained growth

    An S-curve in stock growth typically shows:
    1. Initial slow growth or consolidation phase
    2. Followed by a rapid growth phase that sustains for multiple years
    3. Then stabilization at a higher level

    This function identifies the start of the sustained rapid growth phase (point 2)
    """
    if len(price_series) < 3 or len(years) < 3:
        return np.nan, np.nan

    # Convert to numpy arrays for processing
    prices = np.array(price_series)
    years_array = np.array(years)

    # Calculate year-over-year percentage changes
    pct_changes = np.zeros(len(prices))
    for i in range(1, len(prices)):
        if prices[i-1] > 0:  # Avoid division by zero
            pct_changes[i] = (prices[i] - prices[i-1]) / prices[i-1]

    # Define parameters for S-curve detection
    growth_threshold = 0.15  # 15% minimum annual growth
    sustained_period = min(5, len(prices) - 1)  # Look for 5 years of growth or as much as available
    min_sustained_years = min(3, sustained_period)  # Require at least 3 years to confirm pattern

    # Look for sustained growth periods
    for i in range(1, len(prices) - min_sustained_years + 1):
        # Get the window of percent changes starting at this position
        window = pct_changes[i:i+sustained_period]
        available_years = min(sustained_period, len(window))

        # Check if we have enough positive growth years
        positive_growth_years = sum(window > 0)
        significant_growth_years = sum(window >= growth_threshold)

        # Calculate average growth over this period
        avg_growth = np.mean(window[:available_years])

        # Conditions for S-curve start:
        # 1. Average growth exceeds threshold
        # 2. Most years show positive growth
        # 3. Several years show significant growth
        # 4. Prior period was relatively flat or down (if we have prior data)
        if (avg_growth >= growth_threshold and
            positive_growth_years >= min_sustained_years and
            significant_growth_years >= min_sustained_years * 0.6):

            # Check if there was a change from previous period (if available)
            pattern_confirmed = True
            if i > min_sustained_years:
                prior_period = pct_changes[i-min_sustained_years:i]
                prior_avg_growth = np.mean(prior_period)

                # Confirm we're seeing acceleration from previous period
                if prior_avg_growth >= growth_threshold * 0.8:
                    pattern_confirmed = False

            if pattern_confirmed:
                s_curve_idx = i
                s_curve_price = prices[s_curve_idx]
                s_curve_year = years_array[s_curve_idx]
                return s_curve_price, s_curve_year

    # Alternative approach: Find points where growth rate jumps significantly
    # and stays elevated for multiple years
    for i in range(2, len(pct_changes) - min_sustained_years + 1):
        before_avg = np.mean(pct_changes[max(0, i-2):i])
        after_window = pct_changes[i:i+min_sustained_years]
        after_avg = np.mean(after_window)

        # Check if we have a significant step change in growth rate
        # and the growth is maintained
        if (after_avg > growth_threshold and
            after_avg > before_avg * 2 and  # At least 2x previous growth
            sum(after_window > 0) >= min_sustained_years * 0.8):  # Most years positive

            s_curve_idx = i
            s_curve_price = prices[s_curve_idx]
            s_curve_year = years_array[s_curve_idx]
            return s_curve_price, s_curve_year

    # If we couldn't find a clear S-curve pattern,
    # try to identify a major price bottom followed by recovery
    min_window = min(2, len(prices) - 1)
    for i in range(min_window, len(prices) - min_window):
        # Look for local minimum with clear reversal
        if (prices[i] < np.mean(prices[i-min_window:i]) and
            prices[i] < np.mean(prices[i+1:i+min_window+1]) and
            (prices[i+min_window] / prices[i] - 1) > growth_threshold):

            s_curve_price = prices[i]
            s_curve_year = years_array[i]
            return s_curve_price, s_curve_year

    # If nothing conclusive is found, return NaN
    return np.nan, np.nan

# Function to fetch and process stock data
def fetch_and_process_stock_data(stock_symbol, listing_date, max_retries=3, sleep_time=2):
    """Comprehensive function to fetch and analyze stock data"""
    start_year = listing_date.year
    start_date_str = listing_date.strftime("%Y-%m-%d")

    # Ensure the symbol has the .NS suffix
    full_symbol = stock_symbol if stock_symbol.endswith(".NS") else stock_symbol + ".NS"

    print(f"📥 Processing {full_symbol} from {start_date_str}...")

    try:
        # Download historical stock data from the listing year
        stock_data = safe_download(full_symbol, start=start_date_str, retries=max_retries, delay=sleep_time)

        if stock_data.empty:
            print(f"⚠️ No data found for {full_symbol}")
            return None

        print(f"✅ Data fetched for {full_symbol} ({stock_data.index.min().strftime('%Y-%m-%d')} - {stock_data.index.max().strftime('%Y-%m-%d')})")

        # Check for Close price data
        if 'Close' not in stock_data.columns:
            if 'Adj Close' in stock_data.columns:
                stock_data['Close'] = stock_data['Adj Close']
            else:
                print(f"⚠️ Cannot find price data for {full_symbol}. Skipping.")
                return None

        # Process the data
        stock_data.reset_index(inplace=True)
        stock_data['Year'] = stock_data['Date'].dt.year

        # Add financial year calculation
        stock_data['Fin_Year'] = stock_data['Date'].apply(
            lambda x: x.year if x.month > 3 else x.year - 1
        )

        # Filter out years before listing
        stock_data = stock_data[stock_data['Year'] >= start_year]

        # Yearly metrics - using calendar year for consistency
        yearly_avg_close = stock_data.groupby('Year')['Close'].mean().reset_index()
        yearly_avg_close.columns = ['Year', 'Yearly Average Price']

        # Calculate year-over-year percentage increase in average prices
        yearly_avg_close['Prev Year Avg'] = yearly_avg_close['Yearly Average Price'].shift(1)
        yearly_avg_close['Yearly Avg Price % Increase'] = (
            (yearly_avg_close['Yearly Average Price'] - yearly_avg_close['Prev Year Avg']) /
            yearly_avg_close['Prev Year Avg'] * 100
        )

        # Replace NaN for first year with 0
        yearly_avg_close['Yearly Avg Price % Increase'] = yearly_avg_close['Yearly Avg Price % Increase'].fillna(0)

        # Round to 2 decimal places
        yearly_avg_close['Yearly Avg Price % Increase'] = yearly_avg_close['Yearly Avg Price % Increase'].round(2)

        # Drop the helper column
        yearly_avg_close = yearly_avg_close.drop(columns=['Prev Year Avg'])

        yearly_volatility = stock_data.groupby('Year')['Close'].std().reset_index()
        yearly_volatility.columns = ['Year', 'Yearly Volatility']

        last_dates = stock_data.groupby('Year')['Date'].idxmax()
        last_close_prices = pd.DataFrame()
        last_close_prices['Year'] = stock_data.loc[last_dates, 'Year'].values
        last_close_prices['Market Price'] = stock_data.loc[last_dates, 'Close'].values

        yearly_growth = stock_data.groupby('Year')['Close'].apply(
            lambda x: (x.iloc[-1] / x.iloc[0] - 1) if len(x) > 1 else 0
        ).reset_index()
        yearly_growth.columns = ['Year', 'Yearly Growth']

        # Merge metrics
        merged_data = pd.merge(last_close_prices, yearly_avg_close, on='Year', how='left')
        merged_data = pd.merge(merged_data, yearly_volatility, on='Year', how='left')
        merged_data = pd.merge(merged_data, yearly_growth, on='Year', how='left')
        merged_data.insert(1, 'Stock', full_symbol)

        # Improved S-curve detection
        if len(merged_data) >= 3:  # Need at least 3 years of data for proper detection
            # Sort by year to ensure chronological order
            merged_data = merged_data.sort_values('Year')

            # Get price and year arrays for S-curve detection
            avg_prices = merged_data['Yearly Average Price'].values
            years = merged_data['Year'].values

            # Detect S-curve start
            s_curve_price, s_curve_year = detect_s_curve(avg_prices, years)

            # Calculate years to S-curve
            years_to_scurve = np.nan
            if not np.isnan(s_curve_year):
                years_to_scurve = s_curve_year - start_year

                # Sanity check: if years_to_scurve is negative or too large, something is wrong
                if years_to_scurve < 0 or years_to_scurve > len(merged_data) * 0.75:
                    years_to_scurve = np.nan
                    s_curve_price = np.nan
                    s_curve_year = np.nan
        else:
            s_curve_price = np.nan
            s_curve_year = np.nan
            years_to_scurve = np.nan

        merged_data['Start of S-Curve Price'] = s_curve_price
        merged_data['S-Curve Year'] = s_curve_year
        merged_data['Years to S-Curve Start'] = years_to_scurve

        # Get current financials using the yfinance API
        ticker = yf.Ticker(full_symbol)
        info = safe_info(ticker)

        # Extract current financial metrics
        current_eps = info.get('trailingEps', np.nan)
        current_pe = info.get('trailingPE', np.nan)
        current_book_value = info.get('bookValue', np.nan)
        current_roe = info.get('returnOnEquity', np.nan)

        # Normalize ROE if needed
        if current_roe and current_roe > 1:
            current_roe = current_roe / 100

        # Calculate fundamental PE based on ROE
        fundamental_pe = 1 / current_roe if current_roe and current_roe > 0 else np.nan

        # Add required columns
        for col in ['EPS', 'Projected EPS', 'P/E Ratio', 'Fundamental P/E',
                    'Book Value', 'P/B Ratio', 'ROE', 'Intrinsic Value',
                    'Intrinsic Value (Graham)', 'Intrinsic Value (Combined)',
                    'Market to Intrinsic Ratio']:
            if col not in merged_data.columns:
                merged_data[col] = np.nan

        # Apply derived metrics (approach from first script)
        # Financial metrics - synthesized approach from both scripts
        for index, row in merged_data.iterrows():
            year = row['Year']
            market_price = row['Market Price']
            years_diff = merged_data['Year'].max() - year

            # Estimated EPS based on current EPS with historical growth adjustment
            if current_eps is not None and not pd.isna(current_eps):
                growth_factor = (1 / (1.10 ** years_diff)) if years_diff > 0 else 1
                estimated_eps = current_eps * growth_factor
            else:
                # Fallback calculation from first script
                estimated_eps = (row['Yearly Average Price'] * 0.1 *
                                (1 + row.get('Yearly Growth', 0) * 0.3))
                if not pd.isna(estimated_eps):
                    estimated_eps = max(estimated_eps, 0.01)

            merged_data.at[index, 'EPS'] = estimated_eps

            # Estimated Book Value
            if current_book_value is not None and not pd.isna(current_book_value):
                growth_factor = (1 / (1.08 ** years_diff)) if years_diff > 0 else 1
                estimated_book = current_book_value * growth_factor
            else:
                # Fallback calculation from first script
                factor = (1 - row.get('Yearly Growth', 0) * 0.1 + np.sin(index) * 0.05)
                if not pd.isna(factor):
                    factor = max(factor, 0.05)
                estimated_book = row['Yearly Average Price'] * 0.5 * factor

            merged_data.at[index, 'Book Value'] = estimated_book

            # Calculate P/E and P/B ratios
            if not pd.isna(estimated_eps) and estimated_eps > 0:
                merged_data.at[index, 'P/E Ratio'] = market_price / estimated_eps

            if not pd.isna(estimated_book) and estimated_book > 0:
                merged_data.at[index, 'P/B Ratio'] = market_price / estimated_book

            # Return on Equity
            if current_roe is not None and not pd.isna(current_roe):
                merged_data.at[index, 'ROE'] = current_roe
            else:
                # Calculate from EPS and Book Value
                if not pd.isna(estimated_eps) and not pd.isna(estimated_book) and estimated_book > 0:
                    roe_value = estimated_eps / estimated_book
                    merged_data.at[index, 'ROE'] = roe_value

            # Fundamental P/E calculation
            if not pd.isna(merged_data.at[index, 'ROE']) and merged_data.at[index, 'ROE'] > 0:
                merged_data.at[index, 'Fundamental P/E'] = 1 / merged_data.at[index, 'ROE']
            else:
                # Alternative calculation from first script
                base_pe = 15.0
                if not pd.isna(row.get('Yearly Volatility')) and not pd.isna(row['Yearly Average Price']):
                    base_pe -= (row['Yearly Volatility'] / row['Yearly Average Price'] * 20)

                if not pd.isna(row.get('Yearly Growth')):
                    base_pe += (row['Yearly Growth'] * 10)

                base_pe = max(8, min(25, base_pe))
                merged_data.at[index, 'Fundamental P/E'] = base_pe

            # Calculate future growth and EPS projection
            future_growth_rate = row.get('Yearly Growth', 0)
            if not pd.isna(future_growth_rate):
                future_growth_rate = max(-0.1, min(0.2, future_growth_rate))
                if not pd.isna(estimated_eps):
                    merged_data.at[index, 'Projected EPS'] = estimated_eps * (1 + future_growth_rate)

            # Intrinsic Value calculations
            # Graham's formula
            try:
                if not pd.isna(estimated_eps) and not pd.isna(estimated_book) and estimated_eps > 0 and estimated_book > 0:
                    graham_value = math.sqrt(22.5 * estimated_eps * estimated_book)
                    merged_data.at[index, 'Intrinsic Value (Graham)'] = graham_value
            except:
                pass

            # EPS-based intrinsic value
            if not pd.isna(estimated_eps) and not pd.isna(merged_data.at[index, 'Fundamental P/E']):
                eps_value = estimated_eps * merged_data.at[index, 'Fundamental P/E']
                merged_data.at[index, 'Intrinsic Value'] = eps_value

            # Combined intrinsic value
            values = []
            for col in ['Intrinsic Value', 'Intrinsic Value (Graham)']:
                if col in merged_data.columns and not pd.isna(merged_data.at[index, col]):
                    values.append(merged_data.at[index, col])

            if values:
                combined = np.mean(values)
                merged_data.at[index, 'Intrinsic Value (Combined)'] = combined

                if not pd.isna(market_price) and combined > 0:
                    market_ratio = market_price / combined
                    merged_data.at[index, 'Market to Intrinsic Ratio'] = market_ratio

        # Valuation status
        conditions = [
            merged_data['Market to Intrinsic Ratio'] < 0.8,
            merged_data['Market to Intrinsic Ratio'] < 1.0,
            merged_data['Market to Intrinsic Ratio'] < 1.2,
            merged_data['Market to Intrinsic Ratio'] >= 1.2
        ]
        values = ['Undervalued', 'Fair Value', 'Slightly Overvalued', 'Overvalued']
        merged_data['Valuation Status'] = np.select(conditions, values, default='Fair Value')

        # Round numeric columns
        final_columns = [
            'Year', 'Stock', 'Market Price', 'Yearly Average Price', 'Yearly Avg Price % Increase',
            'Yearly Volatility', 'Yearly Growth', 'EPS', 'Projected EPS', 'P/E Ratio',
            'Fundamental P/E', 'Book Value', 'P/B Ratio', 'ROE', 'Start of S-Curve Price',
            'S-Curve Year', 'Years to S-Curve Start', 'Intrinsic Value',
            'Intrinsic Value (Graham)', 'Intrinsic Value (Combined)',
            'Market to Intrinsic Ratio', 'Valuation Status'
        ]

        # Ensure all columns exist (add if missing)
        for col in final_columns:
            if col not in merged_data.columns and col not in ['Year', 'Stock', 'Valuation Status']:
                merged_data[col] = np.nan

        # Select and order columns that actually exist in the dataframe
        existing_columns = [col for col in final_columns if col in merged_data.columns]
        merged_data = merged_data[existing_columns]

        # Round numeric columns
        for col in merged_data.columns:
            if col not in ['Year', 'Stock', 'Valuation Status', 'S-Curve Year']:
                try:
                    if merged_data[col].dtype in [np.float64, np.float32, float]:
                        merged_data[col] = merged_data[col].round(2)
                except:
                    pass

        # Save to file
        file_path = os.path.join(output_directory, f"{full_symbol}_yearly_analysis.csv")
        merged_data.to_csv(file_path, index=False)

        print(f"📁 Saved: {file_path}")
        return merged_data

    except Exception as error:
        print(f"⚠️ Error processing {full_symbol}: {error}")
        time.sleep(sleep_time)
        return None

# Main execution
all_results = {}
for i, row in stock_df.iterrows():
    symbol = row['SYMBOL']
    listing_date = row['DATE OF LISTING']

    result = fetch_and_process_stock_data(symbol, listing_date)

    if result is not None:
        all_results[symbol] = result

    # Take a break every 10 stocks to avoid rate limits
    if (i + 1) % 10 == 0:
        print("🌍 Taking a longer break to avoid rate limits...")
        time.sleep(15)

# Create summary of latest data for all stocks
if all_results:
    summary_data = []
    for symbol, data in all_results.items():
        if not data.empty:
            latest_year = data['Year'].max()
            latest_data = data[data['Year'] == latest_year].copy()
            if not latest_data.empty:
                summary_data.append(latest_data)

    if summary_data:
        summary_df = pd.concat(summary_data)
        summary_file = os.path.join(output_directory, "all_stocks_latest_analysis.csv")
        summary_df.to_csv(summary_file, index=False)
        print(f"📊 Summary saved to {summary_file}")

print(f"✅ Analysis complete. Output in '{output_directory}/'")

📥 Processing 20MICRONS.NS from 2008-10-06...
YF.download() has changed argument auto_adjust default to True
✅ Data fetched for 20MICRONS.NS (2008-10-06 - 2025-04-11)
📁 Saved: S-Curve_Data/20MICRONS.NS_yearly_analysis.csv
📥 Processing 21STCENMGM.NS from 1995-05-03...
✅ Data fetched for 21STCENMGM.NS (2018-01-11 - 2025-04-11)
📁 Saved: S-Curve_Data/21STCENMGM.NS_yearly_analysis.csv
📥 Processing 360ONE.NS from 2019-09-19...
✅ Data fetched for 360ONE.NS (2019-09-19 - 2025-04-11)
📁 Saved: S-Curve_Data/360ONE.NS_yearly_analysis.csv
📥 Processing 3MINDIA.NS from 2004-08-13...
✅ Data fetched for 3MINDIA.NS (2004-08-13 - 2025-04-11)
📁 Saved: S-Curve_Data/3MINDIA.NS_yearly_analysis.csv
📥 Processing 3PLAND.NS from 1995-07-19...
✅ Data fetched for 3PLAND.NS (2008-03-11 - 2025-04-11)
📁 Saved: S-Curve_Data/3PLAND.NS_yearly_analysis.csv
📥 Processing 5PAISA.NS from 2017-11-16...
✅ Data fetched for 5PAISA.NS (2017-11-16 - 2025-04-11)
📁 Saved: S-Curve_Data/5PAISA.NS_yearly_analysis.csv
📥 Processing 63MOON

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KALYANI.NS']: YFInvalidPeriodError("KALYANI.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KALYANI.NS']: YFInvalidPeriodError("KALYANI.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


📁 Saved: S-Curve_Data/KAKATCEM.NS_yearly_analysis.csv
📥 Processing KALYANI.NS from 2017-02-13...


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['KALYANI.NS']: YFInvalidPeriodError("KALYANI.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


⚠️ No data found for KALYANI.NS
📥 Processing KALYANIFRG.NS from 2007-12-20...
✅ Data fetched for KALYANIFRG.NS (2007-12-20 - 2025-04-11)
📁 Saved: S-Curve_Data/KALYANIFRG.NS_yearly_analysis.csv
📥 Processing KAMATHOTEL.NS from 1996-05-29...
✅ Data fetched for KAMATHOTEL.NS (2002-07-02 - 2025-04-11)
📁 Saved: S-Curve_Data/KAMATHOTEL.NS_yearly_analysis.csv
📥 Processing KAMDHENU.NS from 2006-05-09...
✅ Data fetched for KAMDHENU.NS (2006-05-09 - 2025-04-11)
📁 Saved: S-Curve_Data/KAMDHENU.NS_yearly_analysis.csv
📥 Processing KANANIIND.NS from 2011-12-14...
✅ Data fetched for KANANIIND.NS (2011-12-14 - 2025-04-11)
📁 Saved: S-Curve_Data/KANANIIND.NS_yearly_analysis.csv
📥 Processing KANORICHEM.NS from 1995-02-08...
✅ Data fetched for KANORICHEM.NS (2002-07-01 - 2025-04-11)
📁 Saved: S-Curve_Data/KANORICHEM.NS_yearly_analysis.csv
📥 Processing KANSAINER.NS from 2003-09-15...
✅ Data fetched for KANSAINER.NS (2003-09-15 - 2025-04-11)
📁 Saved: S-Curve_Data/KANSAINER.NS_yearly_analysis.csv
📥 Processing K

In [ ]:
import shutil

# Replace 'your_folder_name' with the name of the folder you want to zip
folder_to_zip = 'S-Curve_Data'
zip_filename = 'S-Curve_Data.zip'

# Create zip file
shutil.make_archive(base_name=zip_filename.replace('.zip', ''), format='zip', root_dir=folder_to_zip)

print(f"Zipped folder saved as: {zip_filename}")


Zipped folder saved as: S-Curve_Data.zip


In [ ]:
import yfinance as yf
import pandas as pd
import os
import numpy as np
import time
import math
from datetime import datetime

# Load stock symbols and their listing dates from CSV
stock_list_path = "NS.csv"  # Path to your CSV
stock_df = pd.read_csv(stock_list_path)
stock_df['DATE OF LISTING'] = pd.to_datetime(stock_df['DATE OF LISTING'])

# Create a directory to store output files (changed output folder name)
output_directory = "Stock_Analysis_Data"
os.makedirs(output_directory, exist_ok=True)

# ---------------------- Rate-Limited Download Functions ----------------------
def safe_download(symbol, start, retries=3, delay=60):
    """Download stock data with rate limit handling"""
    for attempt in range(retries):
        try:
            data = yf.download(symbol, start=start, interval='1d', progress=False)
            if not data.empty:
                return data
        except Exception as e:
            if 'Too Many Requests' in str(e) or 'Rate limited' in str(e):
                print(f"⏳ Rate limit hit for {symbol}. Waiting {delay} seconds (Attempt {attempt+1}/{retries})...")
                time.sleep(delay)
            else:
                print(f"❌ Unexpected error downloading {symbol}: {e}")
                break
    return pd.DataFrame()

def safe_info(ticker_obj, retries=3, delay=30):
    """Fetch ticker info with rate limit handling"""
    for attempt in range(retries):
        try:
            return ticker_obj.info
        except Exception as e:
            if 'Too Many Requests' in str(e) or 'Rate limited' in str(e):
                print(f"⏳ Rate limit (info) hit. Waiting {delay} seconds...")
                time.sleep(delay)
            else:
                print(f"❌ Info error: {e}")
                break
    return {}

# Function to fetch and process stock data
def fetch_and_process_stock_data(stock_symbol, listing_date, max_retries=3, sleep_time=2):
    """Comprehensive function to fetch and analyze stock data"""
    start_year = listing_date.year
    start_date_str = listing_date.strftime("%Y-%m-%d")

    # Ensure the symbol has the .NS suffix
    full_symbol = stock_symbol if stock_symbol.endswith(".NS") else stock_symbol + ".NS"

    print(f"📥 Processing {full_symbol} from {start_date_str}...")

    try:
        # Download historical stock data from the listing year
        stock_data = safe_download(full_symbol, start=start_date_str, retries=max_retries, delay=sleep_time)

        if stock_data.empty:
            print(f"⚠️ No data found for {full_symbol}")
            return None

        print(f"✅ Data fetched for {full_symbol} ({stock_data.index.min().strftime('%Y-%m-%d')} - {stock_data.index.max().strftime('%Y-%m-%d')})")

        # Check for Close price data
        if 'Close' not in stock_data.columns:
            if 'Adj Close' in stock_data.columns:
                stock_data['Close'] = stock_data['Adj Close']
            else:
                print(f"⚠️ Cannot find price data for {full_symbol}. Skipping.")
                return None

        # Process the data
        stock_data.reset_index(inplace=True)
        stock_data['Year'] = stock_data['Date'].dt.year

        # Add financial year calculation
        stock_data['Fin_Year'] = stock_data['Date'].apply(
            lambda x: x.year if x.month > 3 else x.year - 1
        )

        # Filter out years before listing
        stock_data = stock_data[stock_data['Year'] >= start_year]

        # Yearly metrics - using calendar year for consistency
        yearly_avg_close = stock_data.groupby('Year')['Close'].mean().reset_index()
        yearly_avg_close.columns = ['Year', 'Yearly Average Price']

        # Calculate year-over-year percentage increase in average prices
        yearly_avg_close['Prev Year Avg'] = yearly_avg_close['Yearly Average Price'].shift(1)
        yearly_avg_close['Yearly Avg Price % Increase'] = (
            (yearly_avg_close['Yearly Average Price'] - yearly_avg_close['Prev Year Avg']) /
            yearly_avg_close['Prev Year Avg'] * 100
        )

        # Replace NaN for first year with 0
        yearly_avg_close['Yearly Avg Price % Increase'] = yearly_avg_close['Yearly Avg Price % Increase'].fillna(0)

        # Round to 2 decimal places
        yearly_avg_close['Yearly Avg Price % Increase'] = yearly_avg_close['Yearly Avg Price % Increase'].round(2)

        # Drop the helper column
        yearly_avg_close = yearly_avg_close.drop(columns=['Prev Year Avg'])

        yearly_volatility = stock_data.groupby('Year')['Close'].std().reset_index()
        yearly_volatility.columns = ['Year', 'Yearly Volatility']

        last_dates = stock_data.groupby('Year')['Date'].idxmax()
        last_close_prices = pd.DataFrame()
        last_close_prices['Year'] = stock_data.loc[last_dates, 'Year'].values
        last_close_prices['Market Price'] = stock_data.loc[last_dates, 'Close'].values

        yearly_growth = stock_data.groupby('Year')['Close'].apply(
            lambda x: (x.iloc[-1] / x.iloc[0] - 1) if len(x) > 1 else 0
        ).reset_index()
        yearly_growth.columns = ['Year', 'Yearly Growth']

        # Merge metrics
        merged_data = pd.merge(last_close_prices, yearly_avg_close, on='Year', how='left')
        merged_data = pd.merge(merged_data, yearly_volatility, on='Year', how='left')
        merged_data = pd.merge(merged_data, yearly_growth, on='Year', how='left')
        merged_data.insert(1, 'Stock', full_symbol)

        # Get current financials using the yfinance API
        ticker = yf.Ticker(full_symbol)
        info = safe_info(ticker)

        # Extract current financial metrics
        current_eps = info.get('trailingEps', np.nan)
        current_pe = info.get('trailingPE', np.nan)
        current_book_value = info.get('bookValue', np.nan)
        current_roe = info.get('returnOnEquity', np.nan)

        # Normalize ROE if needed
        if current_roe and current_roe > 1:
            current_roe = current_roe / 100

        # Calculate fundamental PE based on ROE
        fundamental_pe = 1 / current_roe if current_roe and current_roe > 0 else np.nan

        # Add required columns
        for col in ['EPS', 'Projected EPS', 'P/E Ratio', 'Fundamental P/E',
                    'Book Value', 'P/B Ratio', 'ROE', 'Intrinsic Value',
                    'Intrinsic Value (Graham)', 'Intrinsic Value (Combined)',
                    'Market to Intrinsic Ratio']:
            if col not in merged_data.columns:
                merged_data[col] = np.nan

        # Apply derived metrics (approach from first script)
        # Financial metrics - synthesized approach from both scripts
        for index, row in merged_data.iterrows():
            year = row['Year']
            market_price = row['Market Price']
            years_diff = merged_data['Year'].max() - year

            # Estimated EPS based on current EPS with historical growth adjustment
            if current_eps is not None and not pd.isna(current_eps):
                growth_factor = (1 / (1.10 ** years_diff)) if years_diff > 0 else 1
                estimated_eps = current_eps * growth_factor
            else:
                # Fallback calculation from first script
                estimated_eps = (row['Yearly Average Price'] * 0.1 *
                                (1 + row.get('Yearly Growth', 0) * 0.3))
                if not pd.isna(estimated_eps):
                    estimated_eps = max(estimated_eps, 0.01)

            merged_data.at[index, 'EPS'] = estimated_eps

            # Estimated Book Value
            if current_book_value is not None and not pd.isna(current_book_value):
                growth_factor = (1 / (1.08 ** years_diff)) if years_diff > 0 else 1
                estimated_book = current_book_value * growth_factor
            else:
                # Fallback calculation from first script
                factor = (1 - row.get('Yearly Growth', 0) * 0.1 + np.sin(index) * 0.05)
                if not pd.isna(factor):
                    factor = max(factor, 0.05)
                estimated_book = row['Yearly Average Price'] * 0.5 * factor

            merged_data.at[index, 'Book Value'] = estimated_book

            # Calculate P/E and P/B ratios
            if not pd.isna(estimated_eps) and estimated_eps > 0:
                merged_data.at[index, 'P/E Ratio'] = market_price / estimated_eps

            if not pd.isna(estimated_book) and estimated_book > 0:
                merged_data.at[index, 'P/B Ratio'] = market_price / estimated_book

            # Return on Equity
            if current_roe is not None and not pd.isna(current_roe):
                merged_data.at[index, 'ROE'] = current_roe
            else:
                # Calculate from EPS and Book Value
                if not pd.isna(estimated_eps) and not pd.isna(estimated_book) and estimated_book > 0:
                    roe_value = estimated_eps / estimated_book
                    merged_data.at[index, 'ROE'] = roe_value

            # Fundamental P/E calculation
            if not pd.isna(merged_data.at[index, 'ROE']) and merged_data.at[index, 'ROE'] > 0:
                merged_data.at[index, 'Fundamental P/E'] = 1 / merged_data.at[index, 'ROE']
            else:
                # Alternative calculation from first script
                base_pe = 15.0
                if not pd.isna(row.get('Yearly Volatility')) and not pd.isna(row['Yearly Average Price']):
                    base_pe -= (row['Yearly Volatility'] / row['Yearly Average Price'] * 20)

                if not pd.isna(row.get('Yearly Growth')):
                    base_pe += (row['Yearly Growth'] * 10)

                base_pe = max(8, min(25, base_pe))
                merged_data.at[index, 'Fundamental P/E'] = base_pe

            # Calculate future growth and EPS projection
            future_growth_rate = row.get('Yearly Growth', 0)
            if not pd.isna(future_growth_rate):
                future_growth_rate = max(-0.1, min(0.2, future_growth_rate))
                if not pd.isna(estimated_eps):
                    merged_data.at[index, 'Projected EPS'] = estimated_eps * (1 + future_growth_rate)

            # Intrinsic Value calculations
            # Graham's formula
            try:
                if not pd.isna(estimated_eps) and not pd.isna(estimated_book) and estimated_eps > 0 and estimated_book > 0:
                    graham_value = math.sqrt(22.5 * estimated_eps * estimated_book)
                    merged_data.at[index, 'Intrinsic Value (Graham)'] = graham_value
            except:
                pass

            # EPS-based intrinsic value
            if not pd.isna(estimated_eps) and not pd.isna(merged_data.at[index, 'Fundamental P/E']):
                eps_value = estimated_eps * merged_data.at[index, 'Fundamental P/E']
                merged_data.at[index, 'Intrinsic Value'] = eps_value

            # Combined intrinsic value
            values = []
            for col in ['Intrinsic Value', 'Intrinsic Value (Graham)']:
                if col in merged_data.columns and not pd.isna(merged_data.at[index, col]):
                    values.append(merged_data.at[index, col])

            if values:
                combined = np.mean(values)
                merged_data.at[index, 'Intrinsic Value (Combined)'] = combined

                if not pd.isna(market_price) and combined > 0:
                    market_ratio = market_price / combined
                    merged_data.at[index, 'Market to Intrinsic Ratio'] = market_ratio

        # Valuation status
        conditions = [
            merged_data['Market to Intrinsic Ratio'] < 0.8,
            merged_data['Market to Intrinsic Ratio'] < 1.0,
            merged_data['Market to Intrinsic Ratio'] < 1.2,
            merged_data['Market to Intrinsic Ratio'] >= 1.2
        ]
        values = ['Undervalued', 'Fair Value', 'Slightly Overvalued', 'Overvalued']
        merged_data['Valuation Status'] = np.select(conditions, values, default='Fair Value')

        # Define final columns (removed S-curve related columns)
        final_columns = [
            'Year', 'Stock', 'Market Price', 'Yearly Average Price', 'Yearly Avg Price % Increase',
            'Yearly Volatility', 'Yearly Growth', 'EPS', 'Projected EPS', 'P/E Ratio',
            'Fundamental P/E', 'Book Value', 'P/B Ratio', 'ROE', 'Intrinsic Value',
            'Intrinsic Value (Graham)', 'Intrinsic Value (Combined)',
            'Market to Intrinsic Ratio', 'Valuation Status'
        ]

        # Ensure all columns exist (add if missing)
        for col in final_columns:
            if col not in merged_data.columns and col not in ['Year', 'Stock', 'Valuation Status']:
                merged_data[col] = np.nan

        # Select and order columns that actually exist in the dataframe
        existing_columns = [col for col in final_columns if col in merged_data.columns]
        merged_data = merged_data[existing_columns]

        # Round numeric columns
        for col in merged_data.columns:
            if col not in ['Year', 'Stock', 'Valuation Status']:
                try:
                    if merged_data[col].dtype in [np.float64, np.float32, float]:
                        merged_data[col] = merged_data[col].round(2)
                except:
                    pass

        # Save to file
        file_path = os.path.join(output_directory, f"{full_symbol}_yearly_analysis.csv")
        merged_data.to_csv(file_path, index=False)

        print(f"📁 Saved: {file_path}")
        return merged_data

    except Exception as error:
        print(f"⚠️ Error processing {full_symbol}: {error}")
        time.sleep(sleep_time)
        return None

# Main execution
all_results = {}
for i, row in stock_df.iterrows():
    symbol = row['SYMBOL']
    listing_date = row['DATE OF LISTING']

    result = fetch_and_process_stock_data(symbol, listing_date)

    if result is not None:
        all_results[symbol] = result

    # Take a break every 10 stocks to avoid rate limits
    if (i + 1) % 10 == 0:
        print("🌍 Taking a longer break to avoid rate limits...")
        time.sleep(15)

# Create summary of latest data for all stocks
if all_results:
    summary_data = []
    for symbol, data in all_results.items():
        if not data.empty:
            latest_year = data['Year'].max()
            latest_data = data[data['Year'] == latest_year].copy()
            if not latest_data.empty:
                summary_data.append(latest_data)

    if summary_data:
        summary_df = pd.concat(summary_data)
        summary_file = os.path.join(output_directory, "all_stocks_latest_analysis.csv")
        summary_df.to_csv(summary_file, index=False)
        print(f"📊 Summary saved to {summary_file}")

print(f"✅ Analysis complete. Output in '{output_directory}/'")

📥 Processing 3IINFOLTD.NS from 2021-10-22...
YF.download() has changed argument auto_adjust default to True
✅ Data fetched for 3IINFOLTD.NS (2021-10-22 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/3IINFOLTD.NS_yearly_analysis.csv
📥 Processing AAATECH.NS from 2022-11-28...
✅ Data fetched for AAATECH.NS (2022-11-28 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/AAATECH.NS_yearly_analysis.csv
📥 Processing AADHARHFC.NS from 2024-05-15...
✅ Data fetched for AADHARHFC.NS (2024-05-15 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/AADHARHFC.NS_yearly_analysis.csv
📥 Processing AAKASH.NS from 2020-09-29...
✅ Data fetched for AAKASH.NS (2020-09-29 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/AAKASH.NS_yearly_analysis.csv
📥 Processing AAREYDRUGS.NS from 2021-08-06...
✅ Data fetched for AAREYDRUGS.NS (2021-08-06 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/AAREYDRUGS.NS_yearly_analysis.csv
📥 Processing AARON.NS from 2020-11-06...
✅ Data fetched for AARON.NS (2020-11-06 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IRIS-RE.NS']: YFInvalidPeriodError("IRIS-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IRIS-RE.NS']: YFInvalidPeriodError("IRIS-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IRIS-RE.NS']: YFInvalidPeriodError("IRIS-RE.NS: Period 'max' is invalid, must be of the format 1d, 5d, etc.")


📁 Saved: Stock_Analysis_Data/IRIS.NS_yearly_analysis.csv
📥 Processing IRIS-RE.NS from 2025-03-27...
⚠️ No data found for IRIS-RE.NS
📥 Processing IRISDOREME.NS from 2021-02-22...
✅ Data fetched for IRISDOREME.NS (2021-02-22 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/IRISDOREME.NS_yearly_analysis.csv
🌍 Taking a longer break to avoid rate limits...
📥 Processing IRMENERGY.NS from 2023-10-26...
✅ Data fetched for IRMENERGY.NS (2023-10-26 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/IRMENERGY.NS_yearly_analysis.csv
📥 Processing ISGEC.NS from 2021-05-05...
✅ Data fetched for ISGEC.NS (2021-05-05 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/ISGEC.NS_yearly_analysis.csv
📥 Processing ISHANCH.NS from 2025-03-24...
✅ Data fetched for ISHANCH.NS (2025-03-24 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/ISHANCH.NS_yearly_analysis.csv
📥 Processing ITCHOTELS.NS from 2025-01-29...
✅ Data fetched for ITCHOTELS.NS (2025-01-29 - 2025-04-15)
📁 Saved: Stock_Analysis_Data/ITCHOTELS.NS_yearly_analysis.csv
📥 Processi

In [ ]:
import shutil

# Replace 'your_folder_name' with the name of the folder you want to zip
folder_to_zip = 'Stock_Analysis_Data'
zip_filename = 'Stock_Analysis_Data.zip'

# Create zip file
shutil.make_archive(base_name=zip_filename.replace('.zip', ''), format='zip', root_dir=folder_to_zip)

print(f"Zipped folder saved as: {zip_filename}")


Zipped folder saved as: Stock_Analysis_Data.zip
